In [14]:
import requests
from selectolax.parser import HTMLParser
from PIL import Image
from io import BytesIO

import asyncio
import aiohttp

In [15]:
url = 'https://es.slideshare.net/odayr1979/assimil-el-nuevo-italiano-sin-esfuerzo-pdf'

In [16]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
}
response = requests.get(url=url, headers=headers)
if response.status_code == 200: parser = HTMLParser(response.text)

In [17]:
imgUrlList = []
slideList = parser.css_first('#slide-container').css('div[class*=slide][data-index]')
for i in range(len(slideList)):
    slide = slideList[i]
    imgUrl = slide.css_first('source').attributes['srcset'].split(',')[-1].strip(' ').split(' ')[0]
    imgUrlList.append(imgUrl)

In [44]:
async def fetchImages(url, session):
    async with session.get(url) as response:
        imgContent = await response.content.read()
        return imgContent

In [46]:
pdfPath = './output/Assimil.pdf'
imgList = []

async with aiohttp.ClientSession() as session:
    tasks = []
    print('Downloading images from slideshare...')
    for imgUrl in imgUrlList:
        tasks.append(asyncio.ensure_future(fetchImages(imgUrl, session)))
    imageBytes = await asyncio.gather(*tasks)
    print('converting images to RGB...')
    for im in imageBytes:
        img = Image.open(BytesIO(im)).convert('RGB')
        imgList.append(img)
    print(f'saving pdf file {pdfPath}')
    imgList[0].save(fp=pdfPath, format='PDF', resolution=100.0, save_all=True, append_images=imgList[1:])


converting images to RGB...
saving pdf file ./output/Assimil.pdf
